In [1]:
# -*- coding: utf-8 -*-
"""
DIC-9345 - Projet 2: Traduction Automatique Neuronale (TAN) - EN->RU
Ce script est adapté pour tourner sur Kaggle avec un accélérateur GPU.
Il configure l'environnement, télécharge les données,
entraîne (fine-tune) un modèle de traduction anglais-russe et l'évalue.
Version configurée pour un TEST RAPIDE (peu de données, 1 époque).
Correction: Réduction de la taille des batchs pour débogage.
Modifications GPU: Rétablissement de la détection/placement GPU explicite et du flag fp16.
"""

# @title 1. Installation des bibliothèques nécessaires
# Installe les bibliothèques Hugging Face (transformers, datasets), SacreBLEU et Accelerate.
!pip install transformers[torch] datasets sacrebleu accelerate evaluate -q

print("Installation terminée.")

# @title 2. Importations et Configuration Initiale
import os
import torch
import numpy as np
# Utilisation de 'evaluate' au lieu de 'load_metric' pour les métriques
from datasets import load_dataset
import evaluate # Nouvelle façon de charger les métriques
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Configuration pour Anglais -> Russe
MODEL_CHECKPOINT = "Helsinki-NLP/opus-mt-en-ru" # Modèle pré-entraîné Anglais -> Russe
SOURCE_LANG = "en"
TARGET_LANG = "ru"
# Utilisation d'opus_books pour la démo, plus petit et plus simple à charger que WMT en-ru
DATASET_NAME = "opus_books"
DATASET_CONFIG = "en-ru"
# Pour utiliser WMT (ex: WMT19), décommentez et ajustez si nécessaire:
# DATASET_NAME = "wmt19"
# DATASET_CONFIG = "ru-en" # WMT utilise souvent la paire inversée dans config

# Limites pour l'exemple (RÉACTIVÉES POUR UN TEST TRÈS RAPIDE)
MAX_TRAIN_SAMPLES = 1000 # Très peu d'exemples pour l'entraînement
MAX_VAL_SAMPLES = 100   # Très peu pour la validation
MAX_TEST_SAMPLES = 100  # Très peu pour le test
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128

# Rétablissement de la vérification explicite du device GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique : {device}")
if torch.cuda.is_available():
    # Afficher le nom du GPU pour confirmer (devrait être T4 si sélectionné)
    print(f"Nom du GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Aucun GPU détecté, utilisation du CPU.")


# @title 3. Chargement et Prétraitement des Données

# Charger le jeu de données (opus_books en-ru)
try:
    raw_datasets_full = load_dataset(DATASET_NAME, DATASET_CONFIG)
    print(f"Dataset {DATASET_NAME} ({DATASET_CONFIG}) chargé.")
    # Inspecter la structure, opus_books a juste 'train'
    print(raw_datasets_full)
    # opus_books n'a qu'un split 'train', nous devons le diviser manuellement
    # Utiliser une plus grande partie pour l'entraînement maintenant
    train_test_split = raw_datasets_full['train'].train_test_split(test_size=0.05, seed=42) # 5% pour test
    train_val_split = train_test_split['train'].train_test_split(test_size=0.05, seed=42) # 5% du reste pour validation (soit ~4.75% du total)

    raw_datasets = {
        'train': train_val_split['train'],
        'validation': train_val_split['test'],
        'test': train_test_split['test']
    }
    print("Dataset divisé en train/validation/test.")
    print(f"Tailles des splits AVANT sélection - Train: {len(raw_datasets['train'])}, Validation: {len(raw_datasets['validation'])}, Test: {len(raw_datasets['test'])}")


except Exception as e:
    print(f"Erreur lors du chargement ou de la division du dataset {DATASET_NAME} ({DATASET_CONFIG}): {e}")
    raise e # Arrêter si le dataset ne peut être chargé

# Charger le tokenizer correspondant au modèle pré-entraîné
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
print(f"Tokenizer chargé pour {MODEL_CHECKPOINT}")

# Fonction de prétraitement pour tokeniser les paires de phrases
def preprocess_function(examples):
    # Accède aux données textuelles correctement, que ce soit via 'translation' ou directement
    if "translation" in examples: # Pour WMT-style datasets
      inputs = [ex[SOURCE_LANG] for ex in examples["translation"]]
      targets = [ex[TARGET_LANG] for ex in examples["translation"]]
    else: # Pour opus_books-style datasets (qui a 'id' et 'translation' comme colonnes)
      # On doit accéder aux langues DANS la colonne 'translation'
      inputs = [ex[SOURCE_LANG] for ex in examples["translation"]]
      targets = [ex[TARGET_LANG] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding=False) # Padding sera géré par DataCollator
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding=False)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Appliquer le prétraitement aux datasets
tokenized_datasets = {}
for split, dataset in raw_datasets.items():
     # CORRECTION: Supprimer les colonnes originales 'id' et 'translation' APRÈS que
     # preprocess_function ait extrait les données et créé les nouvelles colonnes.
     cols_to_remove = ['id', 'translation']
     tokenized_datasets[split] = dataset.map(
         preprocess_function,
         batched=True,
         remove_columns=cols_to_remove, # Utilise la liste corrigée
         desc=f"Tokenizing {split} split..."
     )

print("Prétraitement terminé.")
print("Structure après tokenisation (exemple train):", tokenized_datasets["train"])


# Sélectionner un sous-ensemble (RÉACTIVÉ POUR TEST RAPIDE)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
test_dataset_tokenized = tokenized_datasets["test"]
test_dataset_raw = raw_datasets["test"] # Garder les refs non tokenisées

# Appliquer les limites si elles sont définies et inférieures à la taille actuelle
if MAX_TRAIN_SAMPLES and MAX_TRAIN_SAMPLES < len(train_dataset):
    train_dataset = train_dataset.shuffle(seed=42).select(range(MAX_TRAIN_SAMPLES))
if MAX_VAL_SAMPLES and MAX_VAL_SAMPLES < len(eval_dataset):
     eval_dataset = eval_dataset.shuffle(seed=42).select(range(MAX_VAL_SAMPLES))
     # Assurer que test_dataset_raw correspond si eval est utilisé comme source pour test_dataset_raw plus tard
     # Note: cette logique peut devenir complexe si les splits sources varient.
     # Simplification: on sélectionne aussi dans test_dataset_raw et test_dataset_tokenized
if MAX_TEST_SAMPLES and MAX_TEST_SAMPLES < len(test_dataset_tokenized):
    # Sélectionner les indices d'abord pour garder la cohérence entre raw et tokenized
    indices = list(range(len(test_dataset_tokenized)))
    np.random.seed(42)
    np.random.shuffle(indices)
    selected_indices = indices[:MAX_TEST_SAMPLES]

    test_dataset_tokenized = test_dataset_tokenized.select(selected_indices)
    test_dataset_raw = test_dataset_raw.select(selected_indices)


print(f"Taille du jeu d'entraînement (après sélection): {len(train_dataset)} exemples")
print(f"Taille du jeu d'évaluation (après sélection): {len(eval_dataset)} exemples")
print(f"Taille du jeu de test (après sélection): {len(test_dataset_tokenized)} exemples (tokenisé)")
print(f"Taille du jeu de test (après sélection): {len(test_dataset_raw)} exemples (raw)")


# @title 4. Chargement du Modèle et Configuration de l'Entraînement

# Charger le modèle pré-entraîné Seq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
# Rétablissement de l'envoi explicite au device détecté (devrait être cuda)
model.to(device)
print(f"Modèle {MODEL_CHECKPOINT} chargé et envoyé sur {device}.")

# Nom du run pour le suivi
run_name = f"{MODEL_CHECKPOINT.split('/')[-1]}-finetuned-{SOURCE_LANG}-to-{TARGET_LANG}-quick-test-b4" # Nom pour test rapide batch 4

# Arguments d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir=run_name,
    eval_strategy="epoch", # Ou "steps" avec eval_steps si 1 époque est trop peu pour évaluer
    learning_rate=2e-5,
    per_device_train_batch_size=4, # Taille de batch réduite pour débogage
    per_device_eval_batch_size=4,  # Taille de batch réduite pour débogage
    weight_decay=0.01,
    save_total_limit=1, # Pas besoin de garder beaucoup de checkpoints pour un test
    num_train_epochs=1, # 1 seule époque pour un test rapide
    predict_with_generate=True,
    fp16=torch.cuda.is_available(), # Réactivé: Utiliser la précision mixte si GPU dispo
    push_to_hub=False,
    logging_strategy="steps",
    logging_steps=10, # Logger plus souvent avec peu de données
    save_strategy="epoch", # Sauvegarder à la fin de la seule époque
    load_best_model_at_end=True, # Charger le meilleur (seul) modèle
    metric_for_best_model="bleu",
    greater_is_better=True,
    generation_max_length=MAX_TARGET_LENGTH,
    # report_to="wandb", # Décommenter pour utiliser Weights & Biases
)

# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Métriques d'évaluation (SacreBLEU et chrF)
sacrebleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    bleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels, word_match=True) # chrF++

    result = {"bleu": bleu_result["score"], "chrf++": chrf_result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Initialiser le Trainer
# Il devrait maintenant utiliser le GPU correctement
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Configuration de l'entraînement terminée (mode TEST RAPIDE BATCH 4, prêt pour GPU).")


# @title 5. Lancement de l'Entraînement (Fine-tuning)
# ATTENTION: Devrait être beaucoup plus rapide maintenant.
print("Début de l'entraînement TEST RAPIDE (BATCH 4) sur GPU...")
try:
    train_result = trainer.train()
    trainer.save_model()
    metrics = train_result.metrics
    metrics["train_samples"] = len(train_dataset) # Utilise la taille réelle du dataset après sélection
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    print("Entraînement TEST RAPIDE terminé et modèle sauvegardé.")
except Exception as e:
    print(f"Une erreur est survenue pendant l'entraînement : {e}")
    if "CUDA out of memory" in str(e):
        print("Erreur 'CUDA out of memory'. Essayez de réduire davantage 'per_device_train_batch_size'.")
    print("L'entraînement a été interrompu.")


# @title 6. Évaluation sur le Jeu de Test
print("Début de l'évaluation sur le jeu de test (TEST RAPIDE)...")
model.eval() # Assurer que le modèle est en mode évaluation

# Utiliser trainer.predict() pour une évaluation simplifiée
predict_results = trainer.predict(test_dataset_tokenized, metric_key_prefix="test")

metrics = predict_results.metrics
metrics["test_samples"] = len(test_dataset_raw) # Utilise la taille réelle après sélection

print("----- Résultats de l'évaluation sur le jeu de test (TEST RAPIDE) -----")
print(f"Score SacreBLEU: {metrics.get('test_bleu', 'N/A'):.4f}")
print(f"Score chrF++: {metrics.get('test_chrf++', 'N/A'):.4f}")

# Recalculons pour avoir tous les détails de SacreBLEU si nécessaire
if predict_results.predictions is not None:
    preds = predict_results.predictions
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    references = [ex['translation'][TARGET_LANG] for ex in test_dataset_raw]
    cleaned_preds, cleaned_labels = postprocess_text(decoded_preds, references)

    final_bleu_metric = evaluate.load("sacrebleu")
    test_bleu_results_detailed = final_bleu_metric.compute(predictions=cleaned_preds, references=cleaned_labels)

    print(f"(Recalculé) Score SacreBLEU: {test_bleu_results_detailed['score']:.4f}")
    if 'precisions' in test_bleu_results_detailed:
        print(f"Précisions BLEU (1-4 grams): { [round(p, 4) for p in test_bleu_results_detailed['precisions']] }")
    print(f"Ratio de brièveté (BP): {test_bleu_results_detailed.get('bp', 'N/A'):.4f}")
    print(f"Longueur moyenne des prédictions: {np.mean([len(p.split()) for p in cleaned_preds]):.2f} mots")
    print(f"Longueur moyenne des références: {np.mean([len(l[0].split()) for l in cleaned_labels]):.2f} mots")

    output_prediction_file = os.path.join(run_name, "test_predictions_ru.txt")
    output_reference_file = os.path.join(run_name, "test_references_ru.txt")
    with open(output_prediction_file, "w", encoding="utf-8") as writer:
        writer.write("\n".join(cleaned_preds))
    with open(output_reference_file, "w", encoding="utf-8") as writer:
        writer.write("\n".join([ref[0] for ref in cleaned_labels]))
    print(f"Prédictions sauvegardées dans: {output_prediction_file}")
    print(f"Références sauvegardées dans: {output_reference_file}")

    metrics["test_bleu_precisions"] = test_bleu_results_detailed.get('precisions')
    metrics["test_bp"] = test_bleu_results_detailed.get('bp')


trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

print("Évaluation terminée.")


# @title 7. Exemple d'Inférence (Traduction d'une phrase EN->RU)
# Le modèle devrait être sur le bon device (GPU)
print("\nExemple d'inférence avec le modèle final (TEST RAPIDE)...")
sentence_en = "Machine translation is fascinating."
print(f"Phrase source ({SOURCE_LANG}): {sentence_en}")

# Tokenizer sur CPU, envoi des tenseurs au device du modèle
inputs = tokenizer(sentence_en, return_tensors="pt").to(device) # Envoyer les inputs au device

with torch.no_grad():
    # generate s'exécute sur le device du modèle
    outputs = model.generate(**inputs, max_length=MAX_TARGET_LENGTH, num_beams=4, early_stopping=True)

# outputs sont sur le device, décoder directement
translation_ru = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Traduction ({TARGET_LANG}): {translation_ru}")

# Autre exemple
sentence_en_2 = "This model was trained on the full dataset for more epochs."
print(f"\nPhrase source ({SOURCE_LANG}): {sentence_en_2}")
inputs_2 = tokenizer(sentence_en_2, return_tensors="pt").to(device) # Envoyer les inputs au device
with torch.no_grad():
    outputs_2 = model.generate(**inputs_2, max_length=MAX_TARGET_LENGTH, num_beams=4, early_stopping=True)
translation_ru_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
print(f"Traduction ({TARGET_LANG}): {translation_ru_2}")


print("\nScript TEST RAPIDE terminé pour EN->RU (GPU).")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

2025-04-18 00:48:32.402226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744937312.632741      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744937312.699124      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Utilisation du périphérique : cuda
Nom du GPU: Tesla T4


README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

...

Modèle Helsinki-NLP/opus-mt-en-ru chargé et envoyé sur cuda.


/tmp/ipykernel_19/1483496130.py:228: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Configuration de l'entraînement terminée (mode TEST RAPIDE BATCH 4, prêt pour GPU).
Début de l'entraînement TEST RAPIDE (BATCH 4) sur GPU...
Une erreur est survenue pendant l'entraînement : api_key not configured (no-tty). call wandb.login(key=[your_api_key])
L'entraînement a été interrompu.
Début de l'évaluation sur le jeu de test (TEST RAPIDE)...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TypeError: ChrF._compute() got an unexpected keyword argument 'word_match'